In [1]:
import numpy as np
from matplotlib import pyplot as plt
plt.style.use("solarized-light")

class RGBColor(object):

    def __init__(self, data):
        if isinstance(data, str):
            assert (data[0] == "#") and (len(data) in [ 4, 5, 7, 9 ])
            data = data.lower()
            if len(data) < 6:
                self.__val_r = eval("0x" + data[1]) * 17
                self.__val_g = eval("0x" + data[2]) * 17
                self.__val_b = eval("0x" + data[3]) * 17
            else:
                self.__val_r = eval("0x" + data[1 : 3])
                self.__val_g = eval("0x" + data[3 : 5])
                self.__val_b = eval("0x" + data[5 : 7])
        else:
            assert (0 <= data[0] <= 255) and (0 <= data[1] <= 255) and (0 <= data[2] <= 255)
            self.__val_r, self.__val_g, self.__val_b = data[0], data[1], data[2]

    def __repr__(self) -> str:
        return "RGBColor(%s, %s, %s)" % (self.__val_r, self.__val_g, self.__val_b)

    def __getitem__(self, idx: int) -> float:
        assert isinstance(idx, int) and (0 <= idx < 3)
        return [ self.__val_r, self.__val_g, self.__val_b ][idx]

    @property
    def hexcode(self) -> str:
        return "#%s%s%s" % (hex(round(self.__val_r))[2:], hex(round(self.__val_g))[2:], hex(round(self.__val_b))[2:])

    @property
    def lightenessx2(self) -> float:
        return max(self.__val_r, self.__val_g, self.__val_b) + min(self.__val_r, self.__val_g, self.__val_b)
    @property
    def saturationx2(self) -> float:
        return max(self.__val_r, self.__val_g, self.__val_b) - min(self.__val_r, self.__val_g, self.__val_b)
    @property
    def lighteness(self) -> float:
        return self.lightenessx2 / 2
    @property
    def saturation(self) -> float:
        return self.saturationx2 / 2


In [2]:
def how2lighten(oricolor, newcolor, num_reserve = None, name = None) -> str:
    colorori = RGBColor(oricolor)
    colornew = RGBColor(newcolor)
    if colornew.lightenessx2 > colorori.lightenessx2:
        rate_lighteness = (colornew.lightenessx2 - colorori.lightenessx2) / (510 - colorori.lightenessx2)
        res = "+"
        colortemp = RGBColor([
            #255 * rate_lighteness + colorori[0] * (1 - rate_lighteness),
            (colorori[0] - 255) * (1 - rate_lighteness) + 255,
            (colorori[1] - 255) * (1 - rate_lighteness) + 255,
            (colorori[2] - 255) * (1 - rate_lighteness) + 255,
            ])
    else:
        rate_lighteness = (colorori.lightenessx2 - colornew.lightenessx2) / colorori.lightenessx2
        res = "-"
        colortemp = RGBColor([
            colorori[0] * (1 - rate_lighteness),
            colorori[1] * (1 - rate_lighteness),
            colorori[2] * (1 - rate_lighteness),
            ])

    res = "%s %s %s%%" % (
        name if name else colorori.hexcode,
        res,
        (round(rate_lighteness * (10 ** (num_reserve + 2))) / (10 ** num_reserve)) if num_reserve else (rate_lighteness * 100)
        )
    return res


In [3]:
def how2saturate(oricolor, newcolor, num_reserve = None, name = None) -> str:
    colorori = RGBColor(oricolor)
    colornew = RGBColor(newcolor)
    assert colorori.saturationx2 != 0
    rate_saturation = abs(colornew.saturationx2 - colorori.saturationx2) / colorori.saturationx2
    res = "saturate(%s, %s%%)" % (
        name if name else colorori.hexcode,
        (round(rate_saturation * (10 ** (num_reserve + 2))) / (10 ** num_reserve)) if num_reserve else (rate_saturation * 100)
        )
    if colornew.saturationx2 < colorori.saturationx2:
        res = "de" + res
    return res

In [4]:
solarized_base = {
    "00":   RGBColor("#002b36"),
    "01":   RGBColor("#073642"),
    "02":   RGBColor("#586e75"),
    "03":   RGBColor("#657b83"),
    "04":   RGBColor("#839496"),
    "05":   RGBColor("#93a1a1"),
    "06":   RGBColor("#eee8d5"),
    "07":   RGBColor("#fdf6e3"),
    "yellow":   RGBColor("#b58900"),
    "orange":   RGBColor("#cb4b16"),
    "red":      RGBColor("#dc322f"),
    "magenta":  RGBColor("#d33682"),
    "violet":   RGBColor("#6c71c4"),
    "blue":     RGBColor("#268bd2"),
    "cyan":     RGBColor("#2aa198"),
    "green":    RGBColor("#859900"),
    }

In [9]:
for each in [
    ("02", "#666666"),
    ("04", "#909399"),
    ("04", "#999999"),
    ("05", "#aaaaaa"),
    ("05", "#cccccc"),
    ("05", "#cdcdcd"),
    ("06", "#e6e6e6"),
    ("06", "#e9e9e9"),
    ("06", "#eeeeee"),
    ("06", "#efefef"),
    ("07", "#f4f4f5"),
    ]:
    print(how2lighten(solarized_base[each[0]], each[1], name="$base%s" % each[0]))

$base02 - 0.4878048780487805%
$base04 + 6.986899563318777%
$base04 + 10.91703056768559%
$base05 + 15.841584158415841%
$base05 + 49.504950495049506%
$base05 + 50.495049504950494%
$base06 + 15.254237288135593%
$base06 + 25.423728813559322%
$base06 + 42.3728813559322%
$base06 + 45.76271186440678%
$base07 + 30.0%


In [11]:
for each in [
    ("red", "#fc625d"),
    ("yellow", "#fdbc40"),
    ("green", "#35cd4b"),
    ]:
    print(how2lighten(solarized_base[each[0]], each[1]))
for each in [
    ("#e7728c", "#fc625d"),
    ("#ffd03e", "#fdbc40"),
    ("#1cff03", "#35cd4b"),
    ]:
    print(how2saturate(each[0], each[1]))

#dc322f + 32.098765432098766%
#b5890 + 41.33738601823708%
#85990 + 29.411764705882355%
saturate(#e7728c, 35.8974358974359%)
desaturate(#ffd03e, 2.072538860103627%)
desaturate(#1cff3, 39.682539682539684%)


In [ ]:
for each in [
    ("#e04b49", "#ff2a2a"),
    ("#99caed", "#87daff"),
    ("#1e6da5", "#0684bd"),
    ("#0d3049", "#262a30"),
    ("#e96a36", "#fc6423"),
    ]:
    print(how2saturate(each[0], each[1]))

saturate(#e04b49, 41.05960264900662%)
saturate(#99caed, 42.857142857142854%)
saturate(#1e6da5, 35.55555555555556%)
desaturate(#d3049, 83.33333333333334%)
saturate(#e96a36, 21.22905027932961%)


In [8]:
for each in [
    ("#4c52b7", "#6f42c1"),
    ("#3295da", "#428bca"),
    ("#e0ff15", "#5cb85c"),
    ("#ffd03f", "#f0ad4e"),
    ("#e04a48", "#d9534f"),
    ]:
    print(how2saturate(each[0], each[1]))

saturate(#4c52b7, 18.69158878504673%)
desaturate(#3295da, 19.047619047619047%)
desaturate(#e0ff15, 60.68376068376068%)
desaturate(#ffd03f, 15.625%)
desaturate(#e04a48, 9.210526315789473%)


In [9]:
for each in [
    ("#d3d5ee", "#e1c2ff"),
    ("#b1d7f1", "#b3e5ef"),
    ("#f3ffa5", "#d0e6be"),
    ("#fff2c8", "#fae4cd"),
    ("#f5c4c3", "#ebcdd2"),
    ]:
    print(how2saturate(each[0], each[1]))

saturate(#d3d5ee, 125.92592592592592%)
desaturate(#b1d7f1, 6.25%)
desaturate(#f3ffa5, 55.55555555555556%)
desaturate(#fff2c8, 18.181818181818183%)
desaturate(#f5c4c3, 40.0%)


In [10]:
for each in [
    ("#4c52b7", "#6f42c1"),
    ("#1d6ca3", "#31708f"),
    ("#9bb200", "#3c763d"),
    ("#c59500", "#8a6d3b"),
    ("#c92522", "#a94442"),
    ]:
    print(how2saturate(each[0], each[1]))

saturate(#4c52b7, 18.69158878504673%)
desaturate(#1d6ca3, 29.850746268656714%)
desaturate(#9bb20, 67.41573033707866%)
desaturate(#c5950, 59.89847715736041%)
desaturate(#c92522, 38.32335329341318%)


In [21]:
for each in [
    ("07", "#f9f9f9"),
    ("07", "#f5f5f5"),
    ("06", "#eee"),
    ("05", "#e1e1e1"),
    ("05", "#ddd"),
    ("05", "#ccc"),
    ("05", "#bbb"),
    ("05", "#aaa"),
    ("05", "#999"),
    ("04", "#777"),
    ("02", "#666"),
    ("02", "#555"),
    ("02", "#363636"),
    ("02", "#333"),
    ("02", "#282828"),
    ("01", "#222"),
    ("00", "#111"),
    ]:
    print(how2lighten(solarized_base[each[0]], each[1], name="$base%s" % each[0]))

$base07 + 60.0%
$base07 + 33.33333333333333%
$base06 + 42.3728813559322%
$base05 + 70.29702970297029%
$base05 + 66.33663366336634%
$base05 + 49.504950495049506%
$base05 + 32.67326732673268%
$base05 + 15.841584158415841%
$base05 - 0.6493506493506493%
$base04 - 15.302491103202847%
$base02 - 0.4878048780487805%
$base02 - 17.073170731707318%
$base02 - 47.3170731707317%
$base02 - 50.24390243902439%
$base02 - 60.97560975609756%
$base01 - 6.8493150684931505%
$base00 - 37.03703703703704%


In [5]:
for each in [
    ("red", "#ff2a2a"),
    ("blue", "#87daff"),
    ("blue", "#0684bd"),
    ("blue", "#262a30"),
    ("orange", "#fc6423"),
    ]:
    print(how2lighten(solarized_base[each[0]], each[1], name="$base-%s" % each[0]))

$base-red + 12.345679012345679%
$base-blue + 54.19847328244275%
$base-blue - 21.370967741935484%
$base-blue - 65.32258064516128%
$base-orange + 21.75438596491228%


In [24]:
for each in [
    ("violet", "#6f42c1"),
    ("blue", "#428bca"),
    ("green", "#5cb85c"),
    ("yellow", "#f0ad4e"),
    ("red", "#d9534f"),

    ("violet", "#e1c2ff"),
    ("blue", "#b3e5ef"),
    ("green", "#d0e6be"),
    ("yellow", "#fae4cd"),
    ("red", "#ebcdd2"),

    ("violet", "#6f42c1"),
    ("blue", "#31708f"),
    ("green", "#3c763d"),
    ("yellow", "#8a6d3b"),
    ("red", "#a94442"),
    ]:
    print(how2lighten(solarized_base[each[0]], each[1], name="$base-%s" % each[0]))

$base-violet - 14.802631578947366%
$base-blue + 7.633587786259542%
$base-green + 34.45378151260504%
$base-yellow + 41.641337386018236%
$base-red + 11.934156378600823%
$base-violet + 70.3883495145631%
$base-blue + 64.8854961832061%
$base-green + 74.78991596638656%
$base-yellow + 83.28267477203647%
$base-red + 71.19341563786008%
$base-violet - 14.802631578947366%
$base-blue - 22.58064516129032%
$base-green + 7.002801120448179%
$base-yellow + 4.86322188449848%
$base-red - 11.985018726591761%
